
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [2]:
import sqlite3
import pandas as pd

In [3]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [4]:
# Example for one file
customers = pd.read_csv("data/customers.csv")
customers.to_sql("customers", connection, if_exists="replace", index=False)

1445

In [5]:
# Repeat for all other files in the data folder, or use a loop.
tables = ['data/brands.csv', 'data/categories.csv', 'data/order_items.csv', 'data/orders.csv', 'data/products.csv', 'data/staffs.csv', 'data/stocks.csv', 'data/stores.csv']
for table in tables:
    df = pd.read_csv(table)
    df.to_sql(table.replace('data/', '').replace('.csv', ''), connection, if_exists='replace', index=False)

### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [6]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)


,name
0,customers
1,brands
2,categories
3,order_items
4,orders
5,products
6,staffs
7,stocks
8,stores


## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [7]:

pd.read_sql("SELECT * FROM customers LIMIT 5;", connection)


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,None,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,None,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,None,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,None,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [8]:
pd.read_sql('''SELECT first_name, last_name, city
            From customers
            Order by last_name asc, first_name asc
            LIMIT 10''', connection)

,first_name,last_name,city
0,Ester,Acevedo,San Lorenzo
1,Jamika,Acevedo,Ozone Park
2,Penny,Acevedo,Ballston Spa
3,Bettyann,Acosta,Lancaster
4,Shery,Acosta,Saratoga Springs
5,Corinna,Adams,Rosedale
6,Phylis,Adkins,New York
7,Elinore,Aguilar,San Angelo
8,Janetta,Aguirre,Lancaster
9,Alisia,Albert,Garland


### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [9]:
pd.read_sql('''select product_name, list_price
            from products
            order by list_price desc ''', connection)

,product_name,list_price
0,Trek Domane SLR 9 Disc - 2018,11999.99
1,Trek Domane SLR 8 Disc - 2018,7499.99
2,Trek Silque SLR 8 Women's - 2017,6499.99
3,Trek Domane SL Frameset - 2018,6499.99
4,Trek Domane SL Frameset Women's - 2018,6499.99
...,...,...
316,Trek Kickster - 2018,159.99
317,Trek Boy's Kickster - 2015/2017,149.99
318,Trek Girl's Kickster - 2017,149.99
319,Sun Bicycles Lil Kitt'n - 2017,109.99


### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [10]:
# Your query here
pd.read_sql('''select first_name, last_name, city, state
            from customers
            where state = 'CA'
            order by last_name''', connection)

,first_name,last_name,city,state
0,Ester,Acevedo,San Lorenzo,CA
1,Jamaal,Albert,Torrance,CA
2,Sindy,Anderson,Pomona,CA
3,Twana,Arnold,Anaheim,CA
4,Selene,Austin,Duarte,CA
...,...,...,...,...
279,Darren,Witt,Coachella,CA
280,Lucy,Woods,Palos Verdes Peninsula,CA
281,Joel,Wynn,San Diego,CA
282,Yvone,Yates,San Pablo,CA


### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [11]:
pd.read_sql('''select c.category_name, count(p.product_id) as number_of_products
            from categories c
            join products p on c.category_id = p.category_id
            group by category_name
            order by number_of_products desc;''', connection)

,category_name,number_of_products
0,Cruisers Bicycles,78
1,Road Bikes,60
2,Mountain Bikes,60
3,Children Bicycles,59
4,Comfort Bicycles,30
5,Electric Bikes,24
6,Cyclocross Bicycles,10


### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [12]:
pd.read_sql('''select order_id, order_date, customer_id
            from orders
            where order_date like '2018%'
            order by order_date''', connection)

,order_id,order_date,customer_id
0,1324,2018-01-01,862
1,1325,2018-01-01,68
2,1326,2018-01-01,567
3,1327,2018-01-02,1026
4,1328,2018-01-02,1083
...,...,...,...
287,1611,2018-09-06,6
288,1612,2018-10-21,3
289,1613,2018-11-18,1
290,1614,2018-11-28,135


### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [13]:
pd.read_sql('''select o.order_id, sum(oi.quantity) as total_item_count
            from orders o
            join order_items oi on o.order_id = oi.order_id
            group by o.order_id''', connection)

,order_id,total_item_count
0,1,8
1,2,3
2,3,2
3,4,2
4,5,4
...,...,...
1610,1611,4
1611,1612,8
1612,1613,3
1613,1614,5


### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [14]:
pd.read_sql('''select s.store_name, (oi.quantity * oi.list_price * (1 - oi.discount)) as total_revenue
            from orders o
            join stores s on o.store_id = s.store_id
            join order_items oi on o.order_id = oi.order_id
            group by s.store_name''', connection)

,store_name,total_revenue
0,Baldwin Bikes,557.9907
1,Rowlett Bikes,3193.8810
2,Santa Cruz Bikes,479.9920


### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [18]:
pd.read_sql('''select c.first_name, c.last_name, (oi.quantity * oi.list_price * (1 - oi.discount)) as total_spending
            from orders o
            join customers c on o.customer_id = c.customer_id
            join order_items oi on o.order_id = oi.order_id
            where o.order_status = 4
            order by total_spending desc
            limit 5''', connection)

,first_name,last_name,total_spending
0,Shena,Carter,21599.9820
1,Marlo,Jefferson,12349.9810
2,Shae,Hickman,12349.9810
3,Dorothea,Chang,12349.9810
4,Delmar,Wise,12089.9814


### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [31]:
pd.read_sql('''select c.category_name, p.product_name, oi.quantity
            from orders o
            join order_items oi on o.order_id = oi.order_id
            join products p on oi.product_id = p.product_id
            join categories c on p.category_id = c.category_id
            where o.order_status = 4
            group by c.category_name
            order by oi.quantity desc''',connection)

,category_name,product_name,quantity
0,Road Bikes,Surly Ogre Frameset - 2017,2
1,Electric Bikes,Trek Conduit+ - 2016,2
2,Cyclocross Bicycles,Surly Straggler - 2016,2
3,Cruisers Bicycles,Electra Townie Original 7D EQ - 2016,2
4,Mountain Bikes,Trek Fuel EX 8 29 - 2016,1
5,Comfort Bicycles,Electra Townie Original 7D EQ - 2016,1
6,Children Bicycles,Electra Girl's Hawaii 1 (16-inch) - 2015/2016,1


### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [38]:
pd.read_sql('''select o.staff_id, s.first_name, s.last_name, count(o.staff_id) as order_count
            from orders o
            join staffs s on o.staff_id = s.staff_id
            where o.order_status = 4
            group by o.staff_id
            order by order_count desc''', connection)

,staff_id,first_name,last_name,order_count
0,6,Marcelene,Boyer,521
1,7,Venita,Daniel,498
2,3,Genna,Serrano,147
3,2,Mireya,Copeland,137
4,9,Layla,Terrell,71
5,8,Kali,Vargas,71
